# Декораторы

## Замыкание функций
Вспомним, что такое функции высших порядков.

**Функция высшего порядка** — в программировании функция, принимающая в качестве аргументов другие функции или возвращающая другую функцию в качестве результата. Основная идея состоит в том, что функции имеют тот же статус, что и другие объекты данных.  

Мы можем передавать функции как параметры...
```python
def my_func(inside_func):
    ...
    inside_func()  # Вызов функции принятой в качестве аргумента
    ...
```  

... можем возвращать как результат ...
```python
def a():
    def b(): 
        pass
    return b
```
... и присваивать их другим переменным
```python
def a(): 
    pass

b = a
b()
```

Сделаем функцию, которая будет выполнять принимаемую функцию дважды

In [2]:
def twice_func(inside_func):
    inside_func()
    inside_func()

In [5]:
def hello():
    print("Hello")
    
test = twice_func(hello)

Hello
Hello


Видим что на момент инициализации `twice_func` сразу происходит выполнение функци `hello`.  
Давайте подумаем как можно решить этот вопрос, чтобы мы могли присвоить результат переменной test и после вызывать нашу функцию, которую мы передаем в качестве аргумента?

---
Теперь перейдем к замыканию функций. Разберем что это такое.  
**Замыкание** в программировании — функция, в теле которой присутствуют ссылки на переменные, объявленные вне тела этой функции в окружающем коде и не являющиеся её параметрами. Говоря другим языком, замыкание — функция, которая ссылается на свободные переменные в своём контексте. Не путайте это с глобальными переменными, с котороми может работать функция.


In [9]:
# Реализация с помощью именованных функций:
def make_adder(x):
    def adder(n):
        return x + n # захват переменной "x" из внешнего контекста
    return adder

In [10]:
# функция, которая будет к любому числу прибавлять пятёрку
add_5 = make_adder(5)
print(add_5)

<function make_adder.<locals>.adder at 0x7f8e8a241a60>


In [11]:
print(add_5(10))
print(add_5(100))  

15
105


In [ ]:
# То же самое, но через безымянные функции:
make_adder = lambda x: (
    lambda n: x + n
)

f = make_adder(10)
print(f(5))  # 15
print(f(-1))  # 9

In [ ]:
# пример с функцией которая, возвращает функцию, умножающую на произвольное число
def get_pow(n):
    def pow_(a):
        return a ** n

    def other_pow(b):
        return b * n

    return pow_, other_pow

# Декораторы
Второй паттерн после итератора, который мы изучим с вами это паттерн декоратор.  

Паттерн **декоратор** предназначен для подключения дополнительного поведения к основному объекту, которое будет выполняться до, после или вместо поведения основного объекта.  

Например, хотим включать таймер до вызова функции и выключать после, а потом – выдавать результат.

In [13]:
def my_decorator(a_function_to_decorate):
    # Здесь мы определяем новую функцию – «обертку». Она нам нужна, чтобы выполнять
    # каждый раз при вызове оригинальной функции, а не только один раз
    def wrapper():
        # здесь поместим код, которые будет выполняться до вызова, потом вызов
        # оригинальной функции, потом код после вызова
        print("Я буду выполнен до основного вызова!")
        
        result = a_function_to_decorate()  # не забываем вернуть значение исходной функции
        
        print("Я буду выполнен после основного вызова!")
        return result
    return wrapper

In [14]:
def my_function():
    print("Я – оборачиваемая функция!")
    return 0

my_function = my_decorator(my_function)  # замыкание

In [16]:
result = my_function()

Я буду выполнен до основного вызова!
Я – оборачиваемая функция!
Я буду выполнен после основного вызова!
0


Что теперь будет в переменной `my_function`?  
Функция `my_function`, декоратор `my_decorator` или может быть `wrapper`?

In [17]:
print(my_function)

<function my_decorator.<locals>.wrapper at 0x7f8e8a330598>


In [18]:
print(my_function())

Я буду выполнен до основного вызова!
Я – оборачиваемая функция!
Я буду выполнен после основного вызова!
0


И так будет ~~с каждым~~ при каждом вызове, а не только с на момент инициализации. Одним и тем же декоратором можно обвернуть любую функцию.

---
### Пример декоратора
Давайте реализуем этот паттерн на Python. Поведение декоратора должно выполняться каждый раз, когда вызывается оригинальная функция.  

Нам понадобится функция, которая будет возвращать функцию, внутри которой будет выполняться дополнительное и основное поведение.

In [32]:
# пример с посчетом времени выполнения функции
import time


def decorator_time(fn):
    def wrapper(*args, **kwags):
        print(f"Запустилась функция {fn}")
        t0 = time.time()
        result = fn(*args, **kwags)
        dt = time.time() - t0
        print(f"Функция выполнилась. Время: {dt:.10f}")
        return result
    return wrapper


def pow_2():
    return 10000000 ** 2


def in_build_pow():
    return pow(10000000, 2)


pow_2 = decorator_time(pow_2)
in_build_pow = decorator_time(in_build_pow)

pow_2()
in_build_pow()

Запустилась функция <function pow_2 at 0x7f8e8a241b70>
Функция выполнилась. Время: 0.0000009537
Запустилась функция <function in_build_pow at 0x7f8e8a241f28>
Функция выполнилась. Время: 0.0000021458


100000000000000

---
### Синтаксический сахар
Данная конструкция была настолько часто используема в языке, что ее оформили в качестве синтаксической конструкции

```python
@my_decorator
def my_function():
    pass
```

При этом будет происходить все то же самое, аналогичное
```python
my_function = my_decorator(my_function)
```

In [22]:
# объявляем декоратор
def do_it_twice(func):
    print("Я выполняюсь на момент инициализации")
    def wrapper():
        func()
        func()
    return wrapper

In [23]:
# декорируем функцию
@do_it_twice
def say_whee():
    print("Whee!")

Я выполняюсь на момент инициализации


In [24]:
# вызываем задекорированую функцию
say_whee()

Whee!
Whee!


In [25]:
# какая функция скрывается под переменной say_whee??
print(say_whee)

<function do_it_twice.<locals>.wrapper at 0x7f8e8aa4a730>


---
### Передача аргументов в декорируемую функцию
До этого мы с вами декорировали только функции без параметров.  
А как их передавать?

Попробуем задекорировать функцию, которая умеет принимать параметры

In [26]:
def do_it_twice(func):
    def wrapper():
        func(arg)
        func(arg)
    return wrapper

@do_it_twice
def say_word(word):
    print(word)

say_word("Oo!!!")

TypeError: wrapper() takes 0 positional arguments but 1 was given

Напомним, что декораторы – это лишь обертка над
```python
my_fun = my_decorator(my_fun)
```

И каждый следующий вызов `my_fun` – это вызов `wrapper`. Следовательно, `wrapper` должен уметь принимать параметры и передавать их в вызываемую функцию.

In [27]:
# декоратор, в котором встроенная функция умеет принимать аргументы
def do_it_twice(func):
    def wrapper(arg):
        func(arg)
        func(arg)
    return wrapper

@do_it_twice
def say_word(word):
    print(word)

say_word("Oo!!!")

Oo!!!
Oo!!!


In [28]:
say_word("Hello!!")

Hello!!
Hello!!


---
### Передача переменного количества аргументов в декорируемую функцию
Логичный вопрос – а как передавать параметры, если мы не знаем, что будет вызвано?

In [ ]:
def do_it_twice(func):
    def wrapper(*args, **kwargs):
        func(*args, **kwargs)
        func(*args, **kwargs)
    return wrapper

@do_it_twice
def say_word(word):
    print(word)
    
@do_it_twice
def say_two_words(word1, word2):
    print(f"{word1}, {word2}")

In [ ]:
say_word("Hello")

In [ ]:
say_two_words("Hello", "World")

Чтобы решить этот вопрос нужно использовать **\*args** и **\*\*kwargs**. Для тех кто захочет освежить данную тему в памяти или подробнее её изучить, можно воспользоваться следующим [материалом](https://habr.com/ru/company/ruvds/blog/482464/)

### Порядок декорирования
Порядок декорирования важен!

Т.к. декоратор – синтаксическая обертка, при разном порядке декораторов будут разные результаты обертывания

```python
# Это не одно и тоже
my_fun = wrap1(wrap2(my_fun))
my_fun = wrap2(wrap1(my_fun))
```
Или альтернативный вариант оформления
```python
@wrap1
@wrap2
def my_fun():
    pass

# функция будет иметь другое поведение
@wrap2
@wrap1
def my_fun():
    pass
```

---
Подведем промежуточный итог по декораторам  

Паттерн декоратор нужен для того, чтобы добавлять дополнительное поведение вызываемой функции  

#### Основной шаблон выглядит вот так:
```python
def my_decorator(fn):
    print("Этот код будет выведен в момент декорирования функции")
    def wrapper(*args, **kwargs):
        print('Этот код будет выполняться перед каждым вызовом функции')
        result = fn(*args, **kwargs)
        print('Этот код будет выполняться после каждого вызова функции')
        return result
    return wrapper
```

---
### Фабрика декораторов
А теперь представьте, что мы хотим создать декоратор, который печатает какое-нибудь слово до вызова функции и после.  
А слово хотим задавать в момент декорирования

```python
@printword('Whee!')
def tes(): 
    pass
```

Где можно указать этот параметр?

Давайте возьмем шаблон и попробуем куда-нибудь вставить  этот параметр.
```python
def my_decorator(fn):
    def wrapper(*args, **kwargs):
        result = fn(*args, **kwargs)
        return result
    return wrapper
```

В `wrapper` нельзя – там аргументы вызываемой функции.  
В `my_decorator` нельзя – он принимает на вход единственный аргумент – функцию, которую обертывает.  
Что делать? (подсказка - замыкание функций)

In [ ]:
def my_decorator(fn, arg):
    def wrapper(*args, **kwargs):
        result = fn(*args, **kwargs)
        return result
    return wrapper

Решение – давайте создадим фабрику декораторов – объект, который будет возвращать декоратор.

In [38]:
def decorator_maker(word):
    def my_decorator(fun):
        def wrapper(*ar, **kw):
            print(word)
            print('---')
            res = fun(*ar, **kw)
            print('---')
            print(word)
            return res
        return wrapper
    return my_decorator

@decorator_maker('Hi!')
def f():
    print('Hello')

f()

Hi!
---
Hello
---
Hi!


In [40]:
@decorator_maker('ololo')
def f1():
    print('Hello')

f1()

ololo
---
Hello
---
ololo


В чем отличие от того, что мы разбирали прежде?  

Следует отличать указание декоратора и вызов декоратора
```python
@my_decor
def f():
    pass
```
не идентично 
```python
@my_decor()
def f()
    pass
```
В первом случае вы используете декоратор, во втором – вызывается «создатель» декораторов, который возвращает декоратор

---
Хотим сделать декоратор, который будет взависимости от указанного языка приветствовать пользователя

In [53]:
def welcome(language):  # фабрика декораторов
    welcome_word = {"en": "Hello!", 
                    "ru": "Привет!"}
    def decorator(fn):  # декоратор
        def wrapper(*args, **kwargs):  # задекорированная функция
            print(welcome_word[language])  # приветствие в зависимости от указанного языка в фабрике декораторов
            fn(*args, **kwargs)  # исходная функция
        return wrapper
    return decorator



In [56]:
# welcome('ru')(my_func_ru)()
# welcome -> decorator -> wrapper -> fn
@welcome('ru')
def my_func_ru():
    print("Как дела?")
    
my_func_ru()

Привет!
Как дела?


In [60]:
@welcome('en')
def my_func_en():
    print("How are you?")
    
my_func_en()
print(my_func_en)

Hello!
How are you?
<function welcome.<locals>.decorator.<locals>.wrapper at 0x7f8e8a267b70>


In [59]:
@welcome
def my_func():
    print("How are you?")
    
print(my_func)

<function welcome.<locals>.decorator at 0x7f8e8a2677b8>


### Итоги
Подведем черту под темой декораторов.

- Декораторы добавляют дополнительное поведение функции
- Декораторы – вызовы дополнительных функций, поэтому они немного замедляют ваш код
- Порядок декорирования важен
- Декоратор вызывается один раз, переданные в декоратор параметры изменить больше нельзя.
- Если хотите использовать декораторы, с настраиваемыми параметрами используйте "фабрику декораторов" 

### Где используются декораторы?

1. В самом языке есть @classmethod, @staticmethod и @property – подробнее в курсе ООП
2. Модуль functools:
 - @functools.lru_cache
 - @functools.wraps
3. Flask
 - @app.route('/api/v1/vm', methods=['POST'])
4. Django  
...